<a href="https://colab.research.google.com/github/JimMcDonough/Capstone/blob/main/Roberta_Hyperparameter_tuning_with_lengthandStride.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 20 18:35:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.8 MB/s 
     |████████████████████████████████| 182 kB 85.1 MB/s 
     |████████████████████████████████| 7.6 MB 78.9 MB/s 


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
     |████████████████████████████████| 212 kB 90.2 MB/s 
     |████████████████████████████████| 115 kB 84.0 MB/s 
     |████████████████████████████████| 127 kB 88.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": r"/content/drive/My Drive/sample_data/train.json",
           "validation": r"/content/drive/My Drive/sample_data/val.json",
           "test": r"/content/drive/My Drive/sample_data/test.json"}

In [7]:
features = Features({'answers': [{'text': Value(dtype='string'), 'answer_start': Value(dtype='int32')}],
'context': Value(dtype='string'),
'id': Value(dtype='string'),
'question': Value(dtype='string'),
'title': Value(dtype='string')})

In [8]:
dataset = load_dataset("json", data_files=data_files, features=features, cache_dir= r"/content/drive/My Drive/sample_data/")

  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

data_collator = DefaultDataCollator(return_tensors="tf")

results_list = []
lengths = [100, 384]
strides = [50, 128]
batch_sizes = [16]
epoch_sizes = [3]
learning_rates = [0.000064, 0.000032]

for max_length in lengths:
  for stride in strides:
    for num_epochs in epoch_sizes:
      for batch_size in batch_sizes:
        for learning_rate in learning_rates:
          
          result = {'epochs':num_epochs,
                    'batch_size': batch_size,
                    'learning_rate':learning_rate,
                    'max_length':max_length,
                    'stride': stride}

          def preprocess_function(examples):
              questions = [q.strip() for q in examples["question"]]
              inputs = tokenizer(
                  questions,
                  examples["context"],
                  max_length=max_length,
                  truncation="only_second",  
                  stride = stride,   #only truncates the context nothing else
                  return_overflowing_tokens=True,
                  return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
                  padding="max_length",
              )
              
              offset_mapping = inputs.pop("offset_mapping")
              #print(offset_mapping)
              sample_map = inputs.pop("overflow_to_sample_mapping")
              answers = examples["answers"]
              start_positions = []
              end_positions = []
              #print(answers)
              for i, offset in enumerate(offset_mapping):
                  sample_idx = sample_map[i]
                  answer = answers[sample_idx]
                  #start_char = answer["answer_start"][0]
                  #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
                  start_char = answer[0]["answer_start"]
                  end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
                  sequence_ids = inputs.sequence_ids(i)  #no clue what this does
                  #print(answer, end_char, sequence_ids)
                  # Find the start and end of the context        ; not sure how this section works
                  idx = 0
                  while sequence_ids[idx] != 1:
                      idx += 1
                  context_start = idx
                  while sequence_ids[idx] == 1:
                      idx += 1
                  context_end = idx - 1
              
                  # If the answer is not fully inside the context, label it (0, 0)
                  if offset[context_start][0] > start_char or offset[context_end][1] < end_char: 
                      start_positions.append(0)
                      end_positions.append(0)
                  else:
                      # Otherwise it's the start and end token positions
                      idx = context_start
                      while idx <= context_end and offset[idx][0] <= start_char:
                          idx += 1
                      start_positions.append(idx - 1)
                      
                      idx = context_end
                      while idx >= context_start and offset[idx][1] >= end_char:
                          idx -= 1
                      end_positions.append(idx + 1)
                      
              inputs["start_positions"] = start_positions
              inputs["end_positions"] = end_positions
              return inputs 
              
          tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

          model = TFAutoModelForQuestionAnswering.from_pretrained("roberta-base")
        
          tf_train_set = model.prepare_tf_dataset(
              tokenized_squad["train"],
              shuffle=True,
              batch_size=batch_size,
              collate_fn=data_collator,
          )
          
          tf_validation_set = model.prepare_tf_dataset(
              tokenized_squad["validation"],
              shuffle=False,
              batch_size=batch_size,
              collate_fn=data_collator,
          )
          
          num_train_steps = len(tf_train_set) * num_epochs
          lr_scheduler = PolynomialDecay(
              initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
          )
          
          opt = Adam(learning_rate=lr_scheduler)
          
          loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
          model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
          
          history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)
          result['val_end_accuracy'] = history.history['val_end_logits_accuracy']
          result['val_start_accuracy'] = history.history['val_start_logits_accuracy']
          results_list.append(result) 
          del history
          del model
          del tf_train_set
          del tf_validation_set
          del tokenized_squad





  0%|          | 0/6 [00:00<?, ?ba/s]

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
2878/2878 [==============================] - 180s 58ms/step - loss: 1.6945 - end_logits_loss: 0.8372 - start_logits_loss: 0.8573 - end_logits_accuracy: 0.7666 - start_logits_accuracy: 0.7589 - val_loss: 1.4541 - val_end_logits_loss: 0.7011 - val_start_logits_loss: 0.7530 - val_end_logits_accuracy: 0.7965 - val_start_logits_accuracy: 0.7809
Epoch 2/3
2878/2878 [==============================] - 162s 56ms/step - loss: 0.8257 - end_logits_loss: 0.4072 - start_logits_loss: 0.4186 - end_logits_accuracy: 0.8760 - start_logits_accuracy: 0.8642 - val_loss: 1.6633 - val_end_logits_loss: 0.8079 - val_start_logits_loss: 0.8554 - val_end_logits_accuracy: 0.8010 - val_start_logits_accuracy: 0.7865
Epoch 3/3
2878/2878 [==============================] - 162s 56ms/step - loss: 0.3910 - end_logits_loss: 0.1921 - start_logits_loss: 0.1988 - end_logits_accuracy: 0.9380 - start_logits_accuracy: 0.9321 - val_loss: 1.9184 - val_end_logits_loss: 0.9154 - val_start_logits_loss: 1.0031 - val_end_logi

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
2878/2878 [==============================] - 176s 58ms/step - loss: 1.6296 - end_logits_loss: 0.8023 - start_logits_loss: 0.8273 - end_logits_accuracy: 0.7757 - start_logits_accuracy: 0.7651 - val_loss: 1.4495 - val_end_logits_loss: 0.6938 - val_start_logits_loss: 0.7556 - val_end_logits_accuracy: 0.8027 - val_start_logits_accuracy: 0.7858
Epoch 2/3
2878/2878 [==============================] - 162s 56ms/step - loss: 0.8157 - end_logits_loss: 0.4008 - start_logits_loss: 0.4149 - end_logits_accuracy: 0.8773 - start_logits_accuracy: 0.8652 - val_loss: 1.6897 - val_end_logits_loss: 0.8069 - val_start_logits_loss: 0.8827 - val_end_logits_accuracy: 0.7887 - val_start_logits_accuracy: 0.7699
Epoch 3/3
2878/2878 [==============================] - 162s 56ms/step - loss: 0.4563 - end_logits_loss: 0.2226 - start_logits_loss: 0.2337 - end_logits_accuracy: 0.9272 - start_logits_accuracy: 0.9202 - val_loss: 1.8237 - val_end_logits_loss: 0.8758 - val_start_logits_loss: 0.9479 - val_end_logi

  0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
print(results_list)

In [11]:
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

data_collator = DefaultDataCollator(return_tensors="tf")

results_list = []
lengths = [384]
strides = [128]
batch_sizes = [16]
epoch_sizes = [3]
learning_rates = [0.000064]

for max_length in lengths:
  for stride in strides:
    for num_epochs in epoch_sizes:
      for batch_size in batch_sizes:
        for learning_rate in learning_rates:
          
          result = {'epochs':num_epochs,
                    'batch_size': batch_size,
                    'learning_rate':learning_rate,
                    'max_length':max_length,
                    'stride': stride}

          def preprocess_function(examples):
              questions = [q.strip() for q in examples["question"]]
              inputs = tokenizer(
                  questions,
                  examples["context"],
                  max_length=max_length,
                  truncation="only_second",  
                  stride = stride,   #only truncates the context nothing else
                  return_overflowing_tokens=True,
                  return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
                  padding="max_length",
              )
              
              offset_mapping = inputs.pop("offset_mapping")
              #print(offset_mapping)
              sample_map = inputs.pop("overflow_to_sample_mapping")
              answers = examples["answers"]
              start_positions = []
              end_positions = []
              #print(answers)
              for i, offset in enumerate(offset_mapping):
                  sample_idx = sample_map[i]
                  answer = answers[sample_idx]
                  #start_char = answer["answer_start"][0]
                  #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
                  start_char = answer[0]["answer_start"]
                  end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
                  sequence_ids = inputs.sequence_ids(i)  #no clue what this does
                  #print(answer, end_char, sequence_ids)
                  # Find the start and end of the context        ; not sure how this section works
                  idx = 0
                  while sequence_ids[idx] != 1:
                      idx += 1
                  context_start = idx
                  while sequence_ids[idx] == 1:
                      idx += 1
                  context_end = idx - 1
              
                  # If the answer is not fully inside the context, label it (0, 0)
                  if offset[context_start][0] > start_char or offset[context_end][1] < end_char: 
                      start_positions.append(0)
                      end_positions.append(0)
                  else:
                      # Otherwise it's the start and end token positions
                      idx = context_start
                      while idx <= context_end and offset[idx][0] <= start_char:
                          idx += 1
                      start_positions.append(idx - 1)
                      
                      idx = context_end
                      while idx >= context_start and offset[idx][1] >= end_char:
                          idx -= 1
                      end_positions.append(idx + 1)
                      
              inputs["start_positions"] = start_positions
              inputs["end_positions"] = end_positions
              return inputs 
              
          tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

          model = TFAutoModelForQuestionAnswering.from_pretrained("roberta-base")
        
          tf_train_set = model.prepare_tf_dataset(
              tokenized_squad["train"],
              shuffle=True,
              batch_size=batch_size,
              collate_fn=data_collator,
          )
          
          tf_validation_set = model.prepare_tf_dataset(
              tokenized_squad["validation"],
              shuffle=False,
              batch_size=batch_size,
              collate_fn=data_collator,
          )
          
          num_train_steps = len(tf_train_set) * num_epochs
          lr_scheduler = PolynomialDecay(
              initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
          )
          
          opt = Adam(learning_rate=lr_scheduler)
          
          loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
          model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
          
          history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)
          result['val_end_accuracy'] = history.history['val_end_logits_accuracy']
          result['val_start_accuracy'] = history.history['val_start_logits_accuracy']
          results_list.append(result) 
          #del history
          #del model
          #del tf_train_set
          #del tf_validation_set
          #del tokenized_squad





  0%|          | 0/13 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
796/796 [==============================] - 162s 188ms/step - loss: 2.8758 - end_logits_loss: 1.4094 - start_logits_loss: 1.4665 - end_logits_accuracy: 0.6256 - start_logits_accuracy: 0.6019 - val_loss: 2.3111 - val_end_logits_loss: 1.1009 - val_start_logits_loss: 1.2102 - val_end_logits_accuracy: 0.7066 - val_start_logits_accuracy: 0.6586
Epoch 2/3
796/796 [==============================] - 146s 184ms/step - loss: 1.5263 - end_logits_loss: 0.7466 - start_logits_loss: 0.7796 - end_logits_accuracy: 0.7761 - start_logits_accuracy: 0.7493 - val_loss: 2.4081 - val_end_logits_loss: 1.1576 - val_start_logits_loss: 1.2505 - val_end_logits_accuracy: 0.7172 - val_start_logits_accuracy: 0.6718
Epoch 3/3
796/796 [==============================] - 146s 184ms/step - loss: 0.8934 - end_logits_loss: 0.4279 - start_logits_loss: 0.4655 - end_logits_accuracy: 0.8571 - start_logits_accuracy: 0.8353 - val_loss: 2.5041 - val_end_logits_loss: 1.1744 - val_start_logits_loss: 1.3298 - val_end_logits_